In [0]:
%pip install openai==0.28

In [0]:
%restart_python

GERAÇÃO DO RELATÓRIO PARA FUNDOS FII:

In [0]:
import openai
import os
from pyspark.sql import SparkSession

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')

def cluster_id(cnpj_fundo):
    """
    Retorna o ID do cluster de um fundo da Kinea com base no seu CNPJ.

    Args:
        cnpj_fundo (str): O CNPJ do fundo a ser consultado.

    Returns:
        int or None: O valor da coluna "cluster_id_full" para o CNPJ fornecido,
                     ou None se o fundo não for encontrado.
    """
    spark = SparkSession.builder.getOrCreate()
    
    df = spark.table("desafio_kinea.prospecto_fundos.resultados_cluster_fii")
    
    df_filtrado = df.filter(df.cnpj == cnpj_fundo)
    
    registro = df_filtrado.first()
    
    # Verifica se um registro foi encontrado e retorna o valor da coluna.
    if registro:
        return registro.cluster_id_full
    else:
        return None

def texto_geral_kinea(row):
    # Função que recebe uma linha da tabela que contém informações sobre um fundo e retorna uma descrição textual daquele campo
    
    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em finanças responsável por transformar registros estruturados "
                "de fundos de investimento em descrições textuais claras e objetivas. "
                "Sua tarefa é converter os valores de cada campo recebido em frases coesas, "
                "mantendo contexto financeiro e usando linguagem formal. "
                "Não invente informações; se algum campo estiver vazio ou nulo, ignore-o na narrativa. "
                "Organize o texto em parágrafos corridos, evitando listas e mantendo clareza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Dados do registro:\n\n"
                f"cnpj: {row.cnpj}\n"
                f"data_emissao: {row.data_emissao}\n"
                f"qt_emissoes: {row.qt_emissoes}\n"
                f"nome_fundo: {row.nome_fundo}\n"
                f"tipo_fundo: {row.tipo_fundo}\n"
                f"valor_cota_emissao: {row.valor_cota_emissao}\n"
                f"direito_preferencia_sobras_montante_adicional: {row.direito_preferencia_sobras_montante_adicional}\n"
                f"taxa_distribuicao_emissao: {row.taxa_distribuicao_emissao}\n"
                f"tabela_ativos_fundo: {row.tabela_ativos_fundo}\n"
                f"sumario_experiencia_socios: {row.sumario_experiencia_socios}\n"
                f"quantidade_cotas_emissao: {row.quantidade_cotas_emissao}\n"
                f"quantidade_cotas_adicionais_emissao: {row.quantidade_cotas_adicionais_emissao}\n"
                f"publico_alvo: {row.publico_alvo}\n"
                f"obs_publico_alvo: {row.obs_publico_alvo}\n"
                f"procuracao_AGE: {row.procuracao_AGE}\n"
                f"planilha_custos: {row.planilha_custos}\n"
                f"ordenar_fatores_risco: {row.ordenar_fatores_risco}\n"
                f"montante_minimo_emissao: {row.montante_minimo_emissao}\n"
                f"investimento_minimo_cpf_cnpj: {row.investimento_minimo_cpf_cnpj}\n"
                f"investimento_minimo_inst: {row.investimento_minimo_inst}\n"
                f"investimento_maximo_cpf_cnpj: {row.investimento_maximo_cpf_cnpj}\n"
                f"investimento_maximo_inst: {row.investimento_maximo_inst}\n"
                f"historico_cotacao_bolsa: {row.historico_cotacao_bolsa}\n"
                f"fator_proporcao_dp: {row.fator_proporcao_dp}\n"
                f"diluicao_economica_novas_emissoes: {row.diluicao_economica_novas_emissoes}\n"
                f"criterio_rateio: {row.criterio_rateio}\n"
                f"carteira_fundos_kinea_intrag: {row.carteira_fundos_kinea_intrag}\n"
                f"breve_historico_gestor: {row.breve_historico_gestor}\n"
                f"percentual_oferta_institucional: {row.percentual_oferta_institucional}\n"
                f"volume_base_emissao: {row.volume_base_emissao}\n"
                f"chamada_capital_ipca: {row.chamada_capital_ipca}\n\n"
                f"quantidade_cotas_totais: {row.quantidade_cotas_totais} (emitidas + adicionais)\n"
                f"tipo_anbima: {row.tipo_anbima} (Classificação ANBIMA)\n"
                f"volatilidade_historica: {row.volatilidade_historica} (Volatilidade do Fundo)\n"
                f"liquidez_media: {row.liquidez_media} (facilidade de negociação)\n"
                f"drawdown_max: {row.drawdown_max} (Drawdown em relação ao pico da cotação: essencial para avaliação de risco)\n"
                f"retorno_acumulado: {row.retorno_acumulado} (Retorno Acumulado no Tempo)\n"
                f"sharpe_ratio: {row.sharpe_ratio} (Sharpe-Ratio: retornos ajustado ao risco)\n"
                f"valor_cotado_atual: {row.valor_cotado_atual} (Valor Cotado Atual)\n"
                f"potencial_nova_emissao: {row.potencial_nova_emissao} (se, Valor Cotado Atual > Valor Cota Emissão)\n\n"
                "Gere um texto corrido em parágrafos com essas informações, "
                "mantendo clareza, coesão e foco em contexto financeiro."
            )
        }
    ]
    
    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.1,
    )
    
    texto = response["choices"][0]["message"]["content"].strip()
    return texto

def extrair_texto_fundo(cnpj_fundo):
    spark = SparkSession.builder.getOrCreate()
    df = spark.table("desafio_kinea.prospecto_fundos.resultados_cluster_fii")
    df_filtrado = df.filter(df.cnpj == cnpj_fundo)
    registros = df_filtrado.collect()
    
    if not registros:
        return "Nenhum registro encontrado para o fundo."
    
    return texto_geral_kinea(registros[0])

def texto_cluster(row):
    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em análise de agrupamentos de fundos de investimento. "
                "Sua tarefa é converter as métricas de um cluster em uma descrição textual clara e objetiva. "
                "Crie uma narrativa coesa que contextualize os dados, usando uma linguagem formal e precisa. "
                "Não invente informações; se algum campo estiver vazio ou nulo, ignore-o na narrativa. "
                "Organize o texto em parágrafos corridos, evitando listas e mantendo clareza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Dados do cluster:\n\n"
                f"qtd_fundos: {row.qtd_fundos} (Quantidade de Fundos por Cluster)\n"
                f"media_volume_base_emissao: {row.media_volume_base_emissao} (Volume médio de emissões base por cluster)\n"
                f"media_qt_emissoes: {row.media_qt_emissoes} (Número médio de emissões por cluster)\n"
                f"media_valor_cota_emissao: {row.media_valor_cota_emissao} (Valor médio das cotas de emissão por cluster)\n"
                f"media_taxa_distribuicao_emissao: {row.media_taxa_distribuicao_emissao} (Taxa média de distribuição de emissões por cluster)\n"
                f"media_quantidade_cotas_totais: {row.media_quantidade_cotas_totais} (Número total médio de cotas por cluster)\n"
                f"media_percentual_oferta_institucional: {row.media_percentual_oferta_institucional} (Porcentagem média de ofertas institucionais por cluster)\n"
                f"media_montante_minimo_emissao: {row.media_montante_minimo_emissao} (Montante mínimo médio necessário para emissões por cluster)\n"
                f"std_volume_base_emissao: {row.std_volume_base_emissao} (Desvio padrão do volume de emissões base por cluster)\n"
                f"std_qt_emissoes: {row.std_qt_emissoes} (Desvio padrão do número de emissões por cluster)\n"
                f"std_valor_cota_emissao: {row.std_valor_cota_emissao} (Desvio padrão dos valores de emissão de cotas por cluster)\n"
                f"std_taxa_distribuicao_emissao: {row.std_taxa_distribuicao_emissao} (Desvio padrão das taxas de distribuição por cluster)\n"
                f"std_quantidade_cotas_totais: {row.std_quantidade_cotas_totais} (Desvio padrão do total de cotas por cluster)\n"
                f"std_percentual_oferta_institucional: {row.std_percentual_oferta_institucional} (Desvio padrão das porcentagens de oferta institucional por cluster)\n"
                f"std_montante_minimo_emissao: {row.std_montante_minimo_emissao} (Desvio padrão dos montantes mínimos de emissão por cluster)\n"
                f"proporcao_chamada_capital_ipca: {row.proporcao_chamada_capital_ipca} (Proporção de chamadas de capital ajustadas pela inflação (proporção SIM/NÃO))\n"
                f"media_sharpe_ratio: {row.media_sharpe_ratio} (Média do Sharpe ratio dos fundos por cluster, uma medida de retorno ajustado ao risco)\n"
                f"volume_total_cluster: {row.volume_total_cluster} (Volume total de emissões em todos os fundos do cluster)\n"
                f"idade_media_anos: {row.idade_media_anos} (Idade média dos fundos no cluster em anos)\n"
                f"hhi_volume_base_emissao: {row.hhi_volume_base_emissao} (Herfindahl-Hirschman Index (HHI) para o volume de emissões base, uma medida da concentração de mercado dentro do cluster)\n"
                f"proporcao_participacao_mercado: {row.proporcao_participacao_mercado} (Proporção de participação de mercado, indicando a parcela do cluster no mercado geral para o tipo de fundo)\n\n"
                "Gere um texto corrido em parágrafos com essas informações, "
                "mantendo clareza, coesão e foco em contexto de análise de agrupamentos."
            )
        }
    ]
    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.1,
    )
    
    texto = response["choices"][0]["message"]["content"].strip()
    return texto

def extrair_texto_cluster(cluster_id):
    spark = SparkSession.builder.getOrCreate()
    df = spark.table("desafio_kinea.prospecto_fundos.analise_cluster_fii")
    df_filtrado = df.filter(df.cluster_id_full == cluster_id)
    registros = df_filtrado.collect()
    
    if not registros:
        return "Nenhum registro encontrado para o cluster."
    
    return texto_cluster(registros[0])

def texto_analise_comparativa(row):
    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em análise comparativa de fundos de investimento. "
                "Sua tarefa é converter as métricas de um fundo Kinea e as métricas de comparação com seu cluster "
                "em uma descrição textual clara e concisa. Crie uma narrativa coesa que contextualize as diferenças "
                "e semelhanças, usando linguagem formal e precisa. Não invente informações; "
                "se algum campo estiver vazio ou nulo, ignore-o na narrativa. "
                "Organize o texto em parágrafos corridos, evitando listas e mantendo clareza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Dados comparativos do fundo Kinea:\n\n"
                f"volume_base_emissao: {row.volume_base_emissao} (Volume Base de Emissão do Fundo Kinea)\n"
                f"media_cluster_volume_base_emissao: {row.media_cluster_volume_base_emissao} (Volume base médio de emissões no cluster, excluindo os fundos Kinea)\n"
                f"volume_base_emissao_diff: {row.volume_base_emissao_diff} (Diferença entre o volume base de emissões da Kinea e a média do cluster)\n"
                f"qt_emissoes: {row.qt_emissoes} (Total de emissões do Fundo Kinea)\n"
                f"media_cluster_qt_emissoes: {row.media_cluster_qt_emissoes} (Número médio de emissões no cluster, excluindo os fundos Kinea)\n"
                f"qt_emissoes_diff: {row.qt_emissoes_diff} (Diferença no número de emissões entre o fundo Kinea e a média do cluster)\n"
                f"valor_cota_emissao: {row.valor_cota_emissao} (Valor da Cota de Emissão do Fundo Kinea)\n"
                f"media_cluster_valor_cota_emissao: {row.media_cluster_valor_cota_emissao} (Valor médio da cota de emissão no cluster)\n"
                f"valor_cota_emissao_diff: {row.valor_cota_emissao_diff} (Diferença no valor da cota de emissão entre o fundo Kinea e a média do cluster)\n"
                f"taxa_distribuicao_emissao: {row.taxa_distribuicao_emissao} (Taxa de distribuição de emissão do Fundo Kinea)\n"
                f"media_cluster_taxa_distribuicao_emissao: {row.media_cluster_taxa_distribuicao_emissao} (Taxa média de distribuição de emissão no cluster)\n"
                f"taxa_distribuicao_emissao_diff: {row.taxa_distribuicao_emissao_diff} (Diferença na taxa de distribuição de emissão entre o fundo Kinea e a média do cluster)\n"
                f"quantidade_cotas_totais: {row.quantidade_cotas_totais} (Quantidade total de cotas do Fundo Kinea)\n"
                f"media_cluster_quantidade_cotas_totais: {row.media_cluster_quantidade_cotas_totais} (Quantidade total média de cotas no cluster)\n"
                f"quantidade_cotas_totais_diff: {row.quantidade_cotas_totais_diff} (Diferença na quantidade total de cotas entre o fundo Kinea e a média do cluster)\n"
                f"percentual_oferta_institucional: {row.percentual_oferta_institucional} (Percentual de oferta institucional do Fundo Kinea)\n"
                f"media_cluster_percentual_oferta_institucional: {row.media_cluster_percentual_oferta_institucional} (Percentual médio de oferta institucional no cluster)\n"
                f"percentual_oferta_institucional_diff: {row.percentual_oferta_institucional_diff} (Diferença no percentual de oferta institucional entre o fundo Kinea e a média do cluster)\n"
                f"montante_minimo_emissao: {row.montante_minimo_emissao} (Montante mínimo de emissão do Fundo Kinea)\n"
                f"media_cluster_montante_minimo_emissao: {row.media_cluster_montante_minimo_emissao} (Montante mínimo médio de emissão no cluster)\n"
                f"sharpe_ratio_calc: {row.sharpe_ratio_calc} (Sharpe ratio do Fundo Kinea)\n"
                f"media_cluster_sharpe_ratio_calc: {row.media_cluster_sharpe_ratio_calc} (Sharpe ratio médio no cluster, excluindo os fundos Kinea)\n"
                f"sharpe_ratio_calc_diff: {row.sharpe_ratio_calc_diff} (Diferença nos Sharpe ratios entre o fundo Kinea e a média do cluster)\n"
                f"idade_anos: {row.idade_anos} (Idade do Fundo Kinea em anos)\n"
                f"media_cluster_idade_anos: {row.media_cluster_idade_anos} (Idade média no cluster, excluindo os fundos Kinea)\n"
                f"idade_anos_diff: {row.idade_anos_diff} (Diferença na idade entre o fundo Kinea e a média do cluster)\n"
                f"retorno_acumulado: {row.retorno_acumulado} (Retorno acumulado do Fundo Kinea)\n"
                f"media_cluster_retorno_acumulado: {row.media_cluster_retorno_acumulado} (Retorno acumulado médio no cluster, excluindo os fundos Kinea)\n"
                f"retorno_acumulado_diff: {row.retorno_acumulado_diff} (Diferença nos retornos acumulados entre o fundo Kinea e a média do cluster)\n"
                f"volatilidade_historica: {row.volatilidade_historica} (Volatilidade histórica do Fundo Kinea)\n"
                f"media_cluster_volatilidade_historica: {row.media_cluster_volatilidade_historica} (Volatilidade histórica média no cluster)\n"
                f"volatilidade_historica_diff: {row.volatilidade_historica_diff} (Diferença na volatilidade histórica entre o fundo Kinea e a média do cluster)\n"
                f"volume_total_cluster: {row.volume_total_cluster} (Volume total do Cluster)\n"
                f"market_share_fundo: {row.market_share_fundo} (Market Share da Kinea - Volume Base Emissão/Volume total do Cluster)\n"
                f"dif_market_share: {row.dif_market_share} (Diferença entre o Market Share da Kinea e a Média do Market Share Concorrente)\n\n"
                "Gere um texto corrido que cubra essas informações, "
                "focando na comparação do fundo Kinea com a média do seu cluster."
            )
        }
    ]
    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.1,
    )
    
    texto = response["choices"][0]["message"]["content"].strip()
    return texto

def extrair_texto_analise_comparativa(cnpj_fundo):
    spark = SparkSession.builder.getOrCreate()
    df = spark.table("desafio_kinea.prospecto_fundos.analise_cluster_kinea_individual_fii")
    df_filtrado = df.filter(df.cnpj == cnpj_fundo)
    registros = df_filtrado.collect()
    
    if not registros:
        return "Nenhum registro de análise comparativa encontrado."
    
    return texto_analise_comparativa(registros[0])

def relatorio_economico_final(texto_fundo, texto_cluster, texto_comparativo):
    """
    Combina os três textos em um único prompt para gerar um relatório econômico final.
    """
    messages = [
        {
            "role": "system",
            "content": (
                """Você é um analista financeiro especializado em fundos de investimento e sua tarefa é gerar um relatório estratégico para o time de Business Intelligence da empresa.

            Esse relatório deve ser elaborado a partir de três insumos:

            Descrição do fundo – visão geral sobre objetivos, estratégia, ativos relevantes e perfil de risco.

            Descrição do cluster – explicação do grupo de fundos similares em que o fundo foi alocado, com destaque para padrões de comportamento, oportunidades e ameaças competitivas.

            Análise comparativa – avaliação entre o desempenho do fundo e a média dos fundos do cluster, destacando diferenças relevantes e tendências.

            Estrutura esperada do relatório:

            Título: Nome do fundo analisado.

            Resumo Executivo: Principais achados em até 2 parágrafos, com foco em oportunidades, riscos e relevância para decisões de negócio.

            Visão Geral do Fundo: Contextualização do fundo e seus diferenciais competitivos.

            Cluster e Posição Competitiva: Como o fundo se insere no cluster, padrões relevantes do grupo e implicações estratégicas.

            Insights Comparativos: Principais diferenças entre o fundo e seus pares, destacando fatores de desempenho, custos, volatilidade, consistência, gestão de risco e atratividade.

            Implicações para o Negócio: Pontos que podem apoiar a equipe de BI em:

            Identificação de fundos com maior potencial competitivo;

            Detecção de riscos emergentes;

            Definição de prioridades de monitoramento e alocação de recursos;

            Suporte a estratégias de mercado.

            Conclusão e Recomendações: Síntese dos insights mais acionáveis e possíveis próximos passos.

            Regras de estilo:

            Linguagem clara, objetiva e voltada para extração de insights estratégicos.

            Evitar apenas descrição factual; ressalte implicações de negócio.

            Estruturar o relatório de forma lógica e concisa, com parágrafos bem definidos.

            Sempre destacar o que pode ser útil para tomada de decisão."""
            )
        },
        {
            "role": "user",
            "content": (
                "Com base nos textos a seguir, elabore um relatório econômico completo:\n\n"
                "### 1. Descrição Geral do Fundo\n"
                f"{texto_fundo}\n\n"
                "### 2. Análise do Cluster de Mercado\n"
                f"{texto_cluster}\n\n"
                "### 3. Análise Comparativa Fundo vs. Cluster\n"
                f"{texto_comparativo}\n\n"
            )
        }
    ]

    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.2, # Um pouco mais de temperatura para criatividade no relatório, mas ainda baixa para precisão
    )
    
    relatorio_final = response["choices"][0]["message"]["content"].strip()
    return relatorio_final

def main():
    cnpj_desejado = "42754362000118"
    
    # 1. Extrai o texto geral do fundo
    texto_fundo = extrair_texto_fundo(cnpj_desejado)
    print(f"\n--- Texto Geral do Fundo ---\n{texto_fundo}\n")
    
    # 2. Extrai o texto do cluster
    cluster = cluster_id(cnpj_desejado)
    texto_cluster = extrair_texto_cluster(cluster)
    print(f"\n--- Texto do Cluster ---\n{texto_cluster}\n")

    # 3. Extrai o texto de análise comparativa
    texto_comparativo = extrair_texto_analise_comparativa(cnpj_desejado)
    print(f"\n--- Texto da Análise Comparativa ---\n{texto_comparativo}\n")

    # 4. Combina os textos e gera o relatório final
    if all([texto_fundo, texto_cluster, texto_comparativo]):
        relatorio = relatorio_economico_final(texto_fundo, texto_cluster, texto_comparativo)
        print(f"\n--- Relatório Econômico Final ---\n{relatorio}\n")
    else:
        print("Não foi possível gerar o relatório completo devido a dados ausentes.")

if __name__ == "__main__":
    main()

GERAÇÃO DO RELATÓRIO PARA FUNDOS FIP:

In [0]:
import openai
import os
from pyspark.sql import SparkSession

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')

def cluster_id(cnpj_fundo):
    """
    Retorna o ID do cluster de um fundo da Kinea com base no seu CNPJ.

    Args:
        cnpj_fundo (str): O CNPJ do fundo a ser consultado.

    Returns:
        int or None: O valor da coluna "cluster_id_full" para o CNPJ fornecido,
                     ou None se o fundo não for encontrado.
    """
    spark = SparkSession.builder.getOrCreate()
    
    df = spark.table("desafio_kinea.prospecto_fundos.resultados_cluster_fip")
    
    df_filtrado = df.filter(df.cnpj == cnpj_fundo)
    
    registro = df_filtrado.first()
    
    # Verifica se um registro foi encontrado e retorna o valor da coluna.
    if registro:
        return registro.cluster_id_full
    else:
        return None

def texto_geral_kinea(row):
    # Função que recebe uma linha da tabela que contém informações sobre um fundo e retorna uma descrição textual daquele campo
    
    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em finanças responsável por transformar registros estruturados "
                "de fundos de investimento em descrições textuais claras e objetivas. "
                "Sua tarefa é converter os valores de cada campo recebido em frases coesas, "
                "mantendo contexto financeiro e usando linguagem formal. "
                "Não invente informações; se algum campo estiver vazio ou nulo, ignore-o na narrativa. "
                "Organize o texto em parágrafos corridos, evitando listas e mantendo clareza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Dados do registro:\n\n"
                f"cnpj: {row.cnpj}\n"
                f"data_emissao: {row.data_emissao}\n"
                f"qt_emissoes: {row.qt_emissoes}\n"
                f"nome_fundo: {row.nome_fundo}\n"
                f"tipo_fundo: {row.tipo_fundo}\n"
                f"valor_cota_emissao: {row.valor_cota_emissao}\n"
                f"direito_preferencia_sobras_montante_adicional: {row.direito_preferencia_sobras_montante_adicional}\n"
                f"taxa_distribuicao_emissao: {row.taxa_distribuicao_emissao}\n"
                f"tabela_ativos_fundo: {row.tabela_ativos_fundo}\n"
                f"sumario_experiencia_socios: {row.sumario_experiencia_socios}\n"
                f"quantidade_cotas_emissao: {row.quantidade_cotas_emissao}\n"
                f"quantidade_cotas_adicionais_emissao: {row.quantidade_cotas_adicionais_emissao}\n"
                f"publico_alvo: {row.publico_alvo}\n"
                f"obs_publico_alvo: {row.obs_publico_alvo}\n"
                f"procuracao_AGE: {row.procuracao_AGE}\n"
                f"planilha_custos: {row.planilha_custos}\n"
                f"ordenar_fatores_risco: {row.ordenar_fatores_risco}\n"
                f"montante_minimo_emissao: {row.montante_minimo_emissao}\n"
                f"investimento_minimo_cpf_cnpj: {row.investimento_minimo_cpf_cnpj}\n"
                f"investimento_minimo_inst: {row.investimento_minimo_inst}\n"
                f"investimento_maximo_cpf_cnpj: {row.investimento_maximo_cpf_cnpj}\n"
                f"investimento_maximo_inst: {row.investimento_maximo_inst}\n"
                f"historico_cotacao_bolsa: {row.historico_cotacao_bolsa}\n"
                f"fator_proporcao_dp: {row.fator_proporcao_dp}\n"
                f"diluicao_economica_novas_emissoes: {row.diluicao_economica_novas_emissoes}\n"
                f"criterio_rateio: {row.criterio_rateio}\n"
                f"carteira_fundos_kinea_intrag: {row.carteira_fundos_kinea_intrag}\n"
                f"breve_historico_gestor: {row.breve_historico_gestor}\n"
                f"percentual_oferta_institucional: {row.percentual_oferta_institucional}\n"
                f"volume_base_emissao: {row.volume_base_emissao}\n"
                f"chamada_capital_ipca: {row.chamada_capital_ipca}\n\n"
                f"quantidade_cotas_totais: {row.quantidade_cotas_totais} (emitidas + adicionais)\n"
                f"tipo_anbima: {row.tipo_anbima} (Classificação ANBIMA)\n"
                f"volatilidade_historica: {row.volatilidade_historica} (Volatilidade do Fundo)\n"
                f"liquidez_media: {row.liquidez_media} (facilidade de negociação)\n"
                f"drawdown_max: {row.drawdown_max} (Drawdown em relação ao pico da cotação: essencial para avaliação de risco)\n"
                f"retorno_acumulado: {row.retorno_acumulado} (Retorno Acumulado no Tempo)\n"
                f"sharpe_ratio: {row.sharpe_ratio} (Sharpe-Ratio: retornos ajustado ao risco)\n"
                f"valor_cotado_atual: {row.valor_cotado_atual} (Valor Cotado Atual)\n"
                f"potencial_nova_emissao: {row.potencial_nova_emissao} (se, Valor Cotado Atual > Valor Cota Emissão)\n\n"
                "Gere um texto corrido em parágrafos com essas informações, "
                "mantendo clareza, coesão e foco em contexto financeiro."
            )
        }
    ]
    
    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.1,
    )
    
    texto = response["choices"][0]["message"]["content"].strip()
    return texto

def extrair_texto_fundo(cnpj_fundo):
    spark = SparkSession.builder.getOrCreate()
    df = spark.table("desafio_kinea.prospecto_fundos.resultados_cluster_fip")
    df_filtrado = df.filter(df.cnpj == cnpj_fundo)
    registros = df_filtrado.collect()
    
    if not registros:
        return "Nenhum registro encontrado para o fundo."
    
    return texto_geral_kinea(registros[0])

def texto_cluster(row):
    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em análise de agrupamentos de fundos de investimento. "
                "Sua tarefa é converter as métricas de um cluster em uma descrição textual clara e objetiva. "
                "Crie uma narrativa coesa que contextualize os dados, usando uma linguagem formal e precisa. "
                "Não invente informações; se algum campo estiver vazio ou nulo, ignore-o na narrativa. "
                "Organize o texto em parágrafos corridos, evitando listas e mantendo clareza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Dados do cluster:\n\n"
                f"qtd_fundos: {row.qtd_fundos} (Quantidade de Fundos por Cluster)\n"
                f"media_volume_base_emissao: {row.media_volume_base_emissao} (Volume médio de emissões base por cluster)\n"
                f"media_qt_emissoes: {row.media_qt_emissoes} (Número médio de emissões por cluster)\n"
                f"media_valor_cota_emissao: {row.media_valor_cota_emissao} (Valor médio das cotas de emissão por cluster)\n"
                f"media_taxa_distribuicao_emissao: {row.media_taxa_distribuicao_emissao} (Taxa média de distribuição de emissões por cluster)\n"
                f"media_quantidade_cotas_totais: {row.media_quantidade_cotas_totais} (Número total médio de cotas por cluster)\n"
                f"media_percentual_oferta_institucional: {row.media_percentual_oferta_institucional} (Porcentagem média de ofertas institucionais por cluster)\n"
                f"media_montante_minimo_emissao: {row.media_montante_minimo_emissao} (Montante mínimo médio necessário para emissões por cluster)\n"
                f"std_volume_base_emissao: {row.std_volume_base_emissao} (Desvio padrão do volume de emissões base por cluster)\n"
                f"std_qt_emissoes: {row.std_qt_emissoes} (Desvio padrão do número de emissões por cluster)\n"
                f"std_valor_cota_emissao: {row.std_valor_cota_emissao} (Desvio padrão dos valores de emissão de cotas por cluster)\n"
                f"std_taxa_distribuicao_emissao: {row.std_taxa_distribuicao_emissao} (Desvio padrão das taxas de distribuição por cluster)\n"
                f"std_quantidade_cotas_totais: {row.std_quantidade_cotas_totais} (Desvio padrão do total de cotas por cluster)\n"
                f"std_percentual_oferta_institucional: {row.std_percentual_oferta_institucional} (Desvio padrão das porcentagens de oferta institucional por cluster)\n"
                f"std_montante_minimo_emissao: {row.std_montante_minimo_emissao} (Desvio padrão dos montantes mínimos de emissão por cluster)\n"
                f"proporcao_chamada_capital_ipca: {row.proporcao_chamada_capital_ipca} (Proporção de chamadas de capital ajustadas pela inflação (proporção SIM/NÃO))\n"
                f"media_sharpe_ratio: {row.media_sharpe_ratio} (Média do Sharpe ratio dos fundos por cluster, uma medida de retorno ajustado ao risco)\n"
                f"volume_total_cluster: {row.volume_total_cluster} (Volume total de emissões em todos os fundos do cluster)\n"
                f"idade_media_anos: {row.idade_media_anos} (Idade média dos fundos no cluster em anos)\n"
                f"hhi_volume_base_emissao: {row.hhi_volume_base_emissao} (Herfindahl-Hirschman Index (HHI) para o volume de emissões base, uma medida da concentração de mercado dentro do cluster)\n"
                f"proporcao_participacao_mercado: {row.proporcao_participacao_mercado} (Proporção de participação de mercado, indicando a parcela do cluster no mercado geral para o tipo de fundo)\n\n"
                "Gere um texto corrido em parágrafos com essas informações, "
                "mantendo clareza, coesão e foco em contexto de análise de agrupamentos."
            )
        }
    ]
    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.1,
    )
    
    texto = response["choices"][0]["message"]["content"].strip()
    return texto

def extrair_texto_cluster(cluster_id):
    spark = SparkSession.builder.getOrCreate()
    df = spark.table("desafio_kinea.prospecto_fundos.analise_cluster_fip")
    df_filtrado = df.filter(df.cluster_id_full == cluster_id)
    registros = df_filtrado.collect()
    
    if not registros:
        return "Nenhum registro encontrado para o cluster."
    
    return texto_cluster(registros[0])

def texto_analise_comparativa(row):
    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em análise comparativa de fundos de investimento. "
                "Sua tarefa é converter as métricas de um fundo Kinea e as métricas de comparação com seu cluster "
                "em uma descrição textual clara e concisa. Crie uma narrativa coesa que contextualize as diferenças "
                "e semelhanças, usando linguagem formal e precisa. Não invente informações; "
                "se algum campo estiver vazio ou nulo, ignore-o na narrativa. "
                "Organize o texto em parágrafos corridos, evitando listas e mantendo clareza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Dados comparativos do fundo Kinea:\n\n"
                f"volume_base_emissao: {row.volume_base_emissao} (Volume Base de Emissão do Fundo Kinea)\n"
                f"media_cluster_volume_base_emissao: {row.media_cluster_volume_base_emissao} (Volume base médio de emissões no cluster, excluindo os fundos Kinea)\n"
                f"volume_base_emissao_diff: {row.volume_base_emissao_diff} (Diferença entre o volume base de emissões da Kinea e a média do cluster)\n"
                f"qt_emissoes: {row.qt_emissoes} (Total de emissões do Fundo Kinea)\n"
                f"media_cluster_qt_emissoes: {row.media_cluster_qt_emissoes} (Número médio de emissões no cluster, excluindo os fundos Kinea)\n"
                f"qt_emissoes_diff: {row.qt_emissoes_diff} (Diferença no número de emissões entre o fundo Kinea e a média do cluster)\n"
                f"valor_cota_emissao: {row.valor_cota_emissao} (Valor da Cota de Emissão do Fundo Kinea)\n"
                f"media_cluster_valor_cota_emissao: {row.media_cluster_valor_cota_emissao} (Valor médio da cota de emissão no cluster)\n"
                f"valor_cota_emissao_diff: {row.valor_cota_emissao_diff} (Diferença no valor da cota de emissão entre o fundo Kinea e a média do cluster)\n"
                f"taxa_distribuicao_emissao: {row.taxa_distribuicao_emissao} (Taxa de distribuição de emissão do Fundo Kinea)\n"
                f"media_cluster_taxa_distribuicao_emissao: {row.media_cluster_taxa_distribuicao_emissao} (Taxa média de distribuição de emissão no cluster)\n"
                f"taxa_distribuicao_emissao_diff: {row.taxa_distribuicao_emissao_diff} (Diferença na taxa de distribuição de emissão entre o fundo Kinea e a média do cluster)\n"
                f"quantidade_cotas_totais: {row.quantidade_cotas_totais} (Quantidade total de cotas do Fundo Kinea)\n"
                f"media_cluster_quantidade_cotas_totais: {row.media_cluster_quantidade_cotas_totais} (Quantidade total média de cotas no cluster)\n"
                f"quantidade_cotas_totais_diff: {row.quantidade_cotas_totais_diff} (Diferença na quantidade total de cotas entre o fundo Kinea e a média do cluster)\n"
                f"percentual_oferta_institucional: {row.percentual_oferta_institucional} (Percentual de oferta institucional do Fundo Kinea)\n"
                f"media_cluster_percentual_oferta_institucional: {row.media_cluster_percentual_oferta_institucional} (Percentual médio de oferta institucional no cluster)\n"
                f"percentual_oferta_institucional_diff: {row.percentual_oferta_institucional_diff} (Diferença no percentual de oferta institucional entre o fundo Kinea e a média do cluster)\n"
                f"montante_minimo_emissao: {row.montante_minimo_emissao} (Montante mínimo de emissão do Fundo Kinea)\n"
                f"media_cluster_montante_minimo_emissao: {row.media_cluster_montante_minimo_emissao} (Montante mínimo médio de emissão no cluster)\n"
                f"sharpe_ratio_calc: {row.sharpe_ratio_calc} (Sharpe ratio do Fundo Kinea)\n"
                f"media_cluster_sharpe_ratio_calc: {row.media_cluster_sharpe_ratio_calc} (Sharpe ratio médio no cluster, excluindo os fundos Kinea)\n"
                f"sharpe_ratio_calc_diff: {row.sharpe_ratio_calc_diff} (Diferença nos Sharpe ratios entre o fundo Kinea e a média do cluster)\n"
                f"idade_anos: {row.idade_anos} (Idade do Fundo Kinea em anos)\n"
                f"media_cluster_idade_anos: {row.media_cluster_idade_anos} (Idade média no cluster, excluindo os fundos Kinea)\n"
                f"idade_anos_diff: {row.idade_anos_diff} (Diferença na idade entre o fundo Kinea e a média do cluster)\n"
                f"retorno_acumulado: {row.retorno_acumulado} (Retorno acumulado do Fundo Kinea)\n"
                f"media_cluster_retorno_acumulado: {row.media_cluster_retorno_acumulado} (Retorno acumulado médio no cluster, excluindo os fundos Kinea)\n"
                f"retorno_acumulado_diff: {row.retorno_acumulado_diff} (Diferença nos retornos acumulados entre o fundo Kinea e a média do cluster)\n"
                f"volatilidade_historica: {row.volatilidade_historica} (Volatilidade histórica do Fundo Kinea)\n"
                f"media_cluster_volatilidade_historica: {row.media_cluster_volatilidade_historica} (Volatilidade histórica média no cluster)\n"
                f"volatilidade_historica_diff: {row.volatilidade_historica_diff} (Diferença na volatilidade histórica entre o fundo Kinea e a média do cluster)\n"
                f"volume_total_cluster: {row.volume_total_cluster} (Volume total do Cluster)\n"
                f"market_share_fundo: {row.market_share_fundo} (Market Share da Kinea - Volume Base Emissão/Volume total do Cluster)\n"
                f"dif_market_share: {row.dif_market_share} (Diferença entre o Market Share da Kinea e a Média do Market Share Concorrente)\n\n"
                "Gere um texto corrido que cubra essas informações, "
                "focando na comparação do fundo Kinea com a média do seu cluster."
            )
        }
    ]
    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.1,
    )
    
    texto = response["choices"][0]["message"]["content"].strip()
    return texto

def extrair_texto_analise_comparativa(cnpj_fundo):
    spark = SparkSession.builder.getOrCreate()
    df = spark.table("desafio_kinea.prospecto_fundos.analise_cluster_kinea_individual_fip")
    df_filtrado = df.filter(df.cnpj == cnpj_fundo)
    registros = df_filtrado.collect()
    
    if not registros:
        return "Nenhum registro de análise comparativa encontrado."
    
    return texto_analise_comparativa(registros[0])

def relatorio_economico_final(texto_fundo, texto_cluster, texto_comparativo):
    """
    Combina os três textos em um único prompt para gerar um relatório econômico final.
    """
    messages = [
        {
            "role": "system",
            "content": (
                """Você é um analista financeiro especializado em fundos de investimento e sua tarefa é gerar um relatório estratégico para o time de Business Intelligence da empresa.

            Esse relatório deve ser elaborado a partir de três insumos:

            Descrição do fundo – visão geral sobre objetivos, estratégia, ativos relevantes e perfil de risco.

            Descrição do cluster – explicação do grupo de fundos similares em que o fundo foi alocado, com destaque para padrões de comportamento, oportunidades e ameaças competitivas.

            Análise comparativa – avaliação entre o desempenho do fundo e a média dos fundos do cluster, destacando diferenças relevantes e tendências.

            Estrutura esperada do relatório:

            Título: Nome do fundo analisado.

            Resumo Executivo: Principais achados em até 2 parágrafos, com foco em oportunidades, riscos e relevância para decisões de negócio.

            Visão Geral do Fundo: Contextualização do fundo e seus diferenciais competitivos.

            Cluster e Posição Competitiva: Como o fundo se insere no cluster, padrões relevantes do grupo e implicações estratégicas.

            Insights Comparativos: Principais diferenças entre o fundo e seus pares, destacando fatores de desempenho, custos, volatilidade, consistência, gestão de risco e atratividade.

            Implicações para o Negócio: Pontos que podem apoiar a equipe de BI em:

            Identificação de fundos com maior potencial competitivo;

            Detecção de riscos emergentes;

            Definição de prioridades de monitoramento e alocação de recursos;

            Suporte a estratégias de mercado.

            Conclusão e Recomendações: Síntese dos insights mais acionáveis e possíveis próximos passos.

            Regras de estilo:

            Linguagem clara, objetiva e voltada para extração de insights estratégicos.

            Evitar apenas descrição factual; ressalte implicações de negócio.

            Estruturar o relatório de forma lógica e concisa, com parágrafos bem definidos.

            Sempre destacar o que pode ser útil para tomada de decisão."""
            )
        },
        {
            "role": "user",
            "content": (
                "Com base nos textos a seguir, elabore um relatório econômico completo:\n\n"
                "### 1. Descrição Geral do Fundo\n"
                f"{texto_fundo}\n\n"
                "### 2. Análise do Cluster de Mercado\n"
                f"{texto_cluster}\n\n"
                "### 3. Análise Comparativa Fundo vs. Cluster\n"
                f"{texto_comparativo}\n\n"
            )
        }
    ]

    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.2, # Um pouco mais de temperatura para criatividade no relatório, mas ainda baixa para precisão
    )
    
    relatorio_final = response["choices"][0]["message"]["content"].strip()
    return relatorio_final

def main():
    cnpj_desejado = "57473619000130"
    
    # 1. Extrai o texto geral do fundo
    texto_fundo = extrair_texto_fundo(cnpj_desejado)
    print(f"\n--- Texto Geral do Fundo ---\n{texto_fundo}\n")
    
    # 2. Extrai o texto do cluster
    cluster = cluster_id(cnpj_desejado)
    texto_cluster = extrair_texto_cluster(cluster)
    print(f"\n--- Texto do Cluster ---\n{texto_cluster}\n")

    # 3. Extrai o texto de análise comparativa
    texto_comparativo = extrair_texto_analise_comparativa(cnpj_desejado)
    print(f"\n--- Texto da Análise Comparativa ---\n{texto_comparativo}\n")

    # 4. Combina os textos e gera o relatório final
    if all([texto_fundo, texto_cluster, texto_comparativo]):
        relatorio = relatorio_economico_final(texto_fundo, texto_cluster, texto_comparativo)
        print(f"\n--- Relatório Econômico Final ---\n{relatorio}\n")
    else:
        print("Não foi possível gerar o relatório completo devido a dados ausentes.")

if __name__ == "__main__":
    main()

GERAÇÃO DO RELATÓRIO PARA OS FUNDOS FIDC:

In [0]:
import openai
import os
from pyspark.sql import SparkSession

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')

def cluster_id(cnpj_fundo):
    """
    Retorna o ID do cluster de um fundo da Kinea com base no seu CNPJ.

    Args:
        cnpj_fundo (str): O CNPJ do fundo a ser consultado.

    Returns:
        int or None: O valor da coluna "cluster_id_full" para o CNPJ fornecido,
                     ou None se o fundo não for encontrado.
    """
    spark = SparkSession.builder.getOrCreate()
    
    df = spark.table("desafio_kinea.prospecto_fundos.resultados_cluster_fidc")
    
    df_filtrado = df.filter(df.cnpj == cnpj_fundo)
    
    registro = df_filtrado.first()
    
    # Verifica se um registro foi encontrado e retorna o valor da coluna.
    if registro:
        return registro.cluster_id_full
    else:
        return None

def texto_geral_kinea(row):
    # Função que recebe uma linha da tabela que contém informações sobre um fundo e retorna uma descrição textual daquele campo
    
    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em finanças responsável por transformar registros estruturados "
                "de fundos de investimento em descrições textuais claras e objetivas. "
                "Sua tarefa é converter os valores de cada campo recebido em frases coesas, "
                "mantendo contexto financeiro e usando linguagem formal. "
                "Não invente informações; se algum campo estiver vazio ou nulo, ignore-o na narrativa. "
                "Organize o texto em parágrafos corridos, evitando listas e mantendo clareza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Dados do registro:\n\n"
                f"cnpj: {row.cnpj}\n"
                f"data_emissao: {row.data_emissao}\n"
                f"qt_emissoes: {row.qt_emissoes}\n"
                f"nome_fundo: {row.nome_fundo}\n"
                f"tipo_fundo: {row.tipo_fundo}\n"
                f"valor_cota_emissao: {row.valor_cota_emissao}\n"
                f"direito_preferencia_sobras_montante_adicional: {row.direito_preferencia_sobras_montante_adicional}\n"
                f"taxa_distribuicao_emissao: {row.taxa_distribuicao_emissao}\n"
                f"tabela_ativos_fundo: {row.tabela_ativos_fundo}\n"
                f"sumario_experiencia_socios: {row.sumario_experiencia_socios}\n"
                f"quantidade_cotas_emissao: {row.quantidade_cotas_emissao}\n"
                f"quantidade_cotas_adicionais_emissao: {row.quantidade_cotas_adicionais_emissao}\n"
                f"publico_alvo: {row.publico_alvo}\n"
                f"obs_publico_alvo: {row.obs_publico_alvo}\n"
                f"procuracao_AGE: {row.procuracao_AGE}\n"
                f"planilha_custos: {row.planilha_custos}\n"
                f"ordenar_fatores_risco: {row.ordenar_fatores_risco}\n"
                f"montante_minimo_emissao: {row.montante_minimo_emissao}\n"
                f"investimento_minimo_cpf_cnpj: {row.investimento_minimo_cpf_cnpj}\n"
                f"investimento_minimo_inst: {row.investimento_minimo_inst}\n"
                f"investimento_maximo_cpf_cnpj: {row.investimento_maximo_cpf_cnpj}\n"
                f"investimento_maximo_inst: {row.investimento_maximo_inst}\n"
                f"historico_cotacao_bolsa: {row.historico_cotacao_bolsa}\n"
                f"fator_proporcao_dp: {row.fator_proporcao_dp}\n"
                f"diluicao_economica_novas_emissoes: {row.diluicao_economica_novas_emissoes}\n"
                f"criterio_rateio: {row.criterio_rateio}\n"
                f"carteira_fundos_kinea_intrag: {row.carteira_fundos_kinea_intrag}\n"
                f"breve_historico_gestor: {row.breve_historico_gestor}\n"
                f"percentual_oferta_institucional: {row.percentual_oferta_institucional}\n"
                f"volume_base_emissao: {row.volume_base_emissao}\n"
                f"chamada_capital_ipca: {row.chamada_capital_ipca}\n\n"
                f"quantidade_cotas_totais: {row.quantidade_cotas_totais} (emitidas + adicionais)\n"
                f"tipo_anbima: {row.tipo_anbima} (Classificação ANBIMA)\n"
                f"volatilidade_historica: {row.volatilidade_historica} (Volatilidade do Fundo)\n"
                f"liquidez_media: {row.liquidez_media} (facilidade de negociação)\n"
                f"drawdown_max: {row.drawdown_max} (Drawdown em relação ao pico da cotação: essencial para avaliação de risco)\n"
                f"retorno_acumulado: {row.retorno_acumulado} (Retorno Acumulado no Tempo)\n"
                f"sharpe_ratio: {row.sharpe_ratio} (Sharpe-Ratio: retornos ajustado ao risco)\n"
                f"valor_cotado_atual: {row.valor_cotado_atual} (Valor Cotado Atual)\n"
                f"potencial_nova_emissao: {row.potencial_nova_emissao} (se, Valor Cotado Atual > Valor Cota Emissão)\n\n"
                "Gere um texto corrido em parágrafos com essas informações, "
                "mantendo clareza, coesão e foco em contexto financeiro."
            )
        }
    ]
    
    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.1,
    )
    
    texto = response["choices"][0]["message"]["content"].strip()
    return texto

def extrair_texto_fundo(cnpj_fundo):
    spark = SparkSession.builder.getOrCreate()
    df = spark.table("desafio_kinea.prospecto_fundos.resultados_cluster_fidc")
    df_filtrado = df.filter(df.cnpj == cnpj_fundo)
    registros = df_filtrado.collect()
    
    if not registros:
        return "Nenhum registro encontrado para o fundo."
    
    return texto_geral_kinea(registros[0])

def texto_cluster(row):
    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em análise de agrupamentos de fundos de investimento. "
                "Sua tarefa é converter as métricas de um cluster em uma descrição textual clara e objetiva. "
                "Crie uma narrativa coesa que contextualize os dados, usando uma linguagem formal e precisa. "
                "Não invente informações; se algum campo estiver vazio ou nulo, ignore-o na narrativa. "
                "Organize o texto em parágrafos corridos, evitando listas e mantendo clareza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Dados do cluster:\n\n"
                f"qtd_fundos: {row.qtd_fundos} (Quantidade de Fundos por Cluster)\n"
                f"media_volume_base_emissao: {row.media_volume_base_emissao} (Volume médio de emissões base por cluster)\n"
                f"media_qt_emissoes: {row.media_qt_emissoes} (Número médio de emissões por cluster)\n"
                f"media_valor_cota_emissao: {row.media_valor_cota_emissao} (Valor médio das cotas de emissão por cluster)\n"
                f"media_taxa_distribuicao_emissao: {row.media_taxa_distribuicao_emissao} (Taxa média de distribuição de emissões por cluster)\n"
                f"media_quantidade_cotas_totais: {row.media_quantidade_cotas_totais} (Número total médio de cotas por cluster)\n"
                f"media_percentual_oferta_institucional: {row.media_percentual_oferta_institucional} (Porcentagem média de ofertas institucionais por cluster)\n"
                f"media_montante_minimo_emissao: {row.media_montante_minimo_emissao} (Montante mínimo médio necessário para emissões por cluster)\n"
                f"std_volume_base_emissao: {row.std_volume_base_emissao} (Desvio padrão do volume de emissões base por cluster)\n"
                f"std_qt_emissoes: {row.std_qt_emissoes} (Desvio padrão do número de emissões por cluster)\n"
                f"std_valor_cota_emissao: {row.std_valor_cota_emissao} (Desvio padrão dos valores de emissão de cotas por cluster)\n"
                f"std_taxa_distribuicao_emissao: {row.std_taxa_distribuicao_emissao} (Desvio padrão das taxas de distribuição por cluster)\n"
                f"std_quantidade_cotas_totais: {row.std_quantidade_cotas_totais} (Desvio padrão do total de cotas por cluster)\n"
                f"std_percentual_oferta_institucional: {row.std_percentual_oferta_institucional} (Desvio padrão das porcentagens de oferta institucional por cluster)\n"
                f"std_montante_minimo_emissao: {row.std_montante_minimo_emissao} (Desvio padrão dos montantes mínimos de emissão por cluster)\n"
                f"proporcao_chamada_capital_ipca: {row.proporcao_chamada_capital_ipca} (Proporção de chamadas de capital ajustadas pela inflação (proporção SIM/NÃO))\n"
                f"media_sharpe_ratio: {row.media_sharpe_ratio} (Média do Sharpe ratio dos fundos por cluster, uma medida de retorno ajustado ao risco)\n"
                f"volume_total_cluster: {row.volume_total_cluster} (Volume total de emissões em todos os fundos do cluster)\n"
                f"idade_media_anos: {row.idade_media_anos} (Idade média dos fundos no cluster em anos)\n"
                f"hhi_volume_base_emissao: {row.hhi_volume_base_emissao} (Herfindahl-Hirschman Index (HHI) para o volume de emissões base, uma medida da concentração de mercado dentro do cluster)\n"
                f"proporcao_participacao_mercado: {row.proporcao_participacao_mercado} (Proporção de participação de mercado, indicando a parcela do cluster no mercado geral para o tipo de fundo)\n\n"
                "Gere um texto corrido em parágrafos com essas informações, "
                "mantendo clareza, coesão e foco em contexto de análise de agrupamentos."
            )
        }
    ]
    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.1,
    )
    
    texto = response["choices"][0]["message"]["content"].strip()
    return texto

def extrair_texto_cluster(cluster_id):
    spark = SparkSession.builder.getOrCreate()
    df = spark.table("desafio_kinea.prospecto_fundos.analise_cluster_fidc")
    df_filtrado = df.filter(df.cluster_id_full == cluster_id)
    registros = df_filtrado.collect()
    
    if not registros:
        return "Nenhum registro encontrado para o cluster."
    
    return texto_cluster(registros[0])

def texto_analise_comparativa(row):
    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em análise comparativa de fundos de investimento. "
                "Sua tarefa é converter as métricas de um fundo Kinea e as métricas de comparação com seu cluster "
                "em uma descrição textual clara e concisa. Crie uma narrativa coesa que contextualize as diferenças "
                "e semelhanças, usando linguagem formal e precisa. Não invente informações; "
                "se algum campo estiver vazio ou nulo, ignore-o na narrativa. "
                "Organize o texto em parágrafos corridos, evitando listas e mantendo clareza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Dados comparativos do fundo Kinea:\n\n"
                f"volume_base_emissao: {row.volume_base_emissao} (Volume Base de Emissão do Fundo Kinea)\n"
                f"media_cluster_volume_base_emissao: {row.media_cluster_volume_base_emissao} (Volume base médio de emissões no cluster, excluindo os fundos Kinea)\n"
                f"volume_base_emissao_diff: {row.volume_base_emissao_diff} (Diferença entre o volume base de emissões da Kinea e a média do cluster)\n"
                f"qt_emissoes: {row.qt_emissoes} (Total de emissões do Fundo Kinea)\n"
                f"media_cluster_qt_emissoes: {row.media_cluster_qt_emissoes} (Número médio de emissões no cluster, excluindo os fundos Kinea)\n"
                f"qt_emissoes_diff: {row.qt_emissoes_diff} (Diferença no número de emissões entre o fundo Kinea e a média do cluster)\n"
                f"valor_cota_emissao: {row.valor_cota_emissao} (Valor da Cota de Emissão do Fundo Kinea)\n"
                f"media_cluster_valor_cota_emissao: {row.media_cluster_valor_cota_emissao} (Valor médio da cota de emissão no cluster)\n"
                f"valor_cota_emissao_diff: {row.valor_cota_emissao_diff} (Diferença no valor da cota de emissão entre o fundo Kinea e a média do cluster)\n"
                f"taxa_distribuicao_emissao: {row.taxa_distribuicao_emissao} (Taxa de distribuição de emissão do Fundo Kinea)\n"
                f"media_cluster_taxa_distribuicao_emissao: {row.media_cluster_taxa_distribuicao_emissao} (Taxa média de distribuição de emissão no cluster)\n"
                f"taxa_distribuicao_emissao_diff: {row.taxa_distribuicao_emissao_diff} (Diferença na taxa de distribuição de emissão entre o fundo Kinea e a média do cluster)\n"
                f"quantidade_cotas_totais: {row.quantidade_cotas_totais} (Quantidade total de cotas do Fundo Kinea)\n"
                f"media_cluster_quantidade_cotas_totais: {row.media_cluster_quantidade_cotas_totais} (Quantidade total média de cotas no cluster)\n"
                f"quantidade_cotas_totais_diff: {row.quantidade_cotas_totais_diff} (Diferença na quantidade total de cotas entre o fundo Kinea e a média do cluster)\n"
                f"percentual_oferta_institucional: {row.percentual_oferta_institucional} (Percentual de oferta institucional do Fundo Kinea)\n"
                f"media_cluster_percentual_oferta_institucional: {row.media_cluster_percentual_oferta_institucional} (Percentual médio de oferta institucional no cluster)\n"
                f"percentual_oferta_institucional_diff: {row.percentual_oferta_institucional_diff} (Diferença no percentual de oferta institucional entre o fundo Kinea e a média do cluster)\n"
                f"montante_minimo_emissao: {row.montante_minimo_emissao} (Montante mínimo de emissão do Fundo Kinea)\n"
                f"media_cluster_montante_minimo_emissao: {row.media_cluster_montante_minimo_emissao} (Montante mínimo médio de emissão no cluster)\n"
                f"sharpe_ratio_calc: {row.sharpe_ratio_calc} (Sharpe ratio do Fundo Kinea)\n"
                f"media_cluster_sharpe_ratio_calc: {row.media_cluster_sharpe_ratio_calc} (Sharpe ratio médio no cluster, excluindo os fundos Kinea)\n"
                f"sharpe_ratio_calc_diff: {row.sharpe_ratio_calc_diff} (Diferença nos Sharpe ratios entre o fundo Kinea e a média do cluster)\n"
                f"idade_anos: {row.idade_anos} (Idade do Fundo Kinea em anos)\n"
                f"media_cluster_idade_anos: {row.media_cluster_idade_anos} (Idade média no cluster, excluindo os fundos Kinea)\n"
                f"idade_anos_diff: {row.idade_anos_diff} (Diferença na idade entre o fundo Kinea e a média do cluster)\n"
                f"retorno_acumulado: {row.retorno_acumulado} (Retorno acumulado do Fundo Kinea)\n"
                f"media_cluster_retorno_acumulado: {row.media_cluster_retorno_acumulado} (Retorno acumulado médio no cluster, excluindo os fundos Kinea)\n"
                f"retorno_acumulado_diff: {row.retorno_acumulado_diff} (Diferença nos retornos acumulados entre o fundo Kinea e a média do cluster)\n"
                f"volatilidade_historica: {row.volatilidade_historica} (Volatilidade histórica do Fundo Kinea)\n"
                f"media_cluster_volatilidade_historica: {row.media_cluster_volatilidade_historica} (Volatilidade histórica média no cluster)\n"
                f"volatilidade_historica_diff: {row.volatilidade_historica_diff} (Diferença na volatilidade histórica entre o fundo Kinea e a média do cluster)\n"
                f"volume_total_cluster: {row.volume_total_cluster} (Volume total do Cluster)\n"
                f"market_share_fundo: {row.market_share_fundo} (Market Share da Kinea - Volume Base Emissão/Volume total do Cluster)\n"
                f"dif_market_share: {row.dif_market_share} (Diferença entre o Market Share da Kinea e a Média do Market Share Concorrente)\n\n"
                "Gere um texto corrido que cubra essas informações, "
                "focando na comparação do fundo Kinea com a média do seu cluster."
            )
        }
    ]
    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.1,
    )
    
    texto = response["choices"][0]["message"]["content"].strip()
    return texto

def extrair_texto_analise_comparativa(cnpj_fundo):
    spark = SparkSession.builder.getOrCreate()
    df = spark.table("desafio_kinea.prospecto_fundos.analise_cluster_kinea_individual_fidc")
    df_filtrado = df.filter(df.cnpj == cnpj_fundo)
    registros = df_filtrado.collect()
    
    if not registros:
        return "Nenhum registro de análise comparativa encontrado."
    
    return texto_analise_comparativa(registros[0])

def relatorio_economico_final(texto_fundo, texto_cluster, texto_comparativo):
    """
    Combina os três textos em um único prompt para gerar um relatório econômico final.
    """
    messages = [
        {
            "role": "system",
            "content": (
                """Você é um analista financeiro especializado em fundos de investimento e sua tarefa é gerar um relatório estratégico para o time de Business Intelligence da empresa.

            Esse relatório deve ser elaborado a partir de três insumos:

            Descrição do fundo – visão geral sobre objetivos, estratégia, ativos relevantes e perfil de risco.

            Descrição do cluster – explicação do grupo de fundos similares em que o fundo foi alocado, com destaque para padrões de comportamento, oportunidades e ameaças competitivas.

            Análise comparativa – avaliação entre o desempenho do fundo e a média dos fundos do cluster, destacando diferenças relevantes e tendências.

            Estrutura esperada do relatório:

            Título: Nome do fundo analisado.

            Resumo Executivo: Principais achados em até 2 parágrafos, com foco em oportunidades, riscos e relevância para decisões de negócio.

            Visão Geral do Fundo: Contextualização do fundo e seus diferenciais competitivos.

            Cluster e Posição Competitiva: Como o fundo se insere no cluster, padrões relevantes do grupo e implicações estratégicas.

            Insights Comparativos: Principais diferenças entre o fundo e seus pares, destacando fatores de desempenho, custos, volatilidade, consistência, gestão de risco e atratividade.

            Implicações para o Negócio: Pontos que podem apoiar a equipe de BI em:

            Identificação de fundos com maior potencial competitivo;

            Detecção de riscos emergentes;

            Definição de prioridades de monitoramento e alocação de recursos;

            Suporte a estratégias de mercado.

            Conclusão e Recomendações: Síntese dos insights mais acionáveis e possíveis próximos passos.

            Regras de estilo:

            Linguagem clara, objetiva e voltada para extração de insights estratégicos.

            Evitar apenas descrição factual; ressalte implicações de negócio.

            Estruturar o relatório de forma lógica e concisa, com parágrafos bem definidos.

            Sempre destacar o que pode ser útil para tomada de decisão."""
            )
        },
        {
            "role": "user",
            "content": (
                "Com base nos textos a seguir, elabore um relatório econômico completo:\n\n"
                "### 1. Descrição Geral do Fundo\n"
                f"{texto_fundo}\n\n"
                "### 2. Análise do Cluster de Mercado\n"
                f"{texto_cluster}\n\n"
                "### 3. Análise Comparativa Fundo vs. Cluster\n"
                f"{texto_comparativo}\n\n"
            )
        }
    ]

    response = openai.ChatCompletion.create(
        engine="gpt-4.1-mini",
        messages=messages,
        temperature=0.2, # Um pouco mais de temperatura para criatividade no relatório, mas ainda baixa para precisão
    )
    
    relatorio_final = response["choices"][0]["message"]["content"].strip()
    return relatorio_final

def main():
    cnpj_desejado = "26324298000189"
    
    # 1. Extrai o texto geral do fundo
    texto_fundo = extrair_texto_fundo(cnpj_desejado)
    print(f"\n--- Texto Geral do Fundo ---\n{texto_fundo}\n")
    
    # 2. Extrai o texto do cluster
    cluster = cluster_id(cnpj_desejado)
    texto_cluster = extrair_texto_cluster(cluster)
    print(f"\n--- Texto do Cluster ---\n{texto_cluster}\n")

    # 3. Extrai o texto de análise comparativa
    texto_comparativo = extrair_texto_analise_comparativa(cnpj_desejado)
    print(f"\n--- Texto da Análise Comparativa ---\n{texto_comparativo}\n")

    # 4. Combina os textos e gera o relatório final
    if all([texto_fundo, texto_cluster, texto_comparativo]):
        relatorio = relatorio_economico_final(texto_fundo, texto_cluster, texto_comparativo)
        print(f"\n--- Relatório Econômico Final ---\n{relatorio}\n")
    else:
        print("Não foi possível gerar o relatório completo devido a dados ausentes.")

if __name__ == "__main__":
    main()